In [1]:
import pandas as pd
import pyodbc
import numpy as np

class get_df:

    def __init__(self):
        self.DB_infos = {"X01" : ("192.168.1.16", "X01", "cm23357868", "86875332")}
        self.querys = {}
        self.db = str()
        self.df_dict =  dict()
        self.w_code_dict = dict()
        self.point_dict = dict()
        self.result_dict = dict()
        
    def db_connect(self, DB_info):
        DB_Host, DB_Name, DB_User, DB_Password = self.DB_infos[DB_info]
        print("Connecting to database using pyodbc...")
        self.db = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server}; SERVER=' + DB_Host + '; DATABASE=' + DB_Name + '; UID=' + DB_User + '; PWD=' + DB_Password)
        print("Succesfully Connected to database using pyodbc!")
      
    def load_data(self, query):
        df =  pd.read_sql(self.querys[query], self.db)
        return df
    @staticmethod
    def make_CB():
        df ="""SELECT    [日期]
                        ,[代號]
                        ,[名稱]
                        ,[開盤價]
                        ,[最高價]
                        ,[最低價]
                        ,[收盤價]
                        ,[漲跌]
                        ,[漲幅(%)]
                        ,[成交張數]
                        ,[成交筆數]
                        ,[成交金額(千)]
                        ,[成交均價]
                        ,[次日參考價]
                        ,[標的代號]
                        ,[標的名稱]
                        ,[標的收盤價]
                        ,[標的漲跌幅(%)]
                        ,[最高價(議價)]
                        ,[最低價(議價)]
                        ,[收盤價(議價)]
                        ,[漲跌(議價)]
                        ,[漲幅(議價)(%)]
                        ,[成交張數(議價)]
                        ,[成交筆數(議價)]
                        ,[成交金額(議價)(千)]
                        ,[RTIME]
                        ,[開盤價(成交)]
                        ,[最高價(成交)]
                        ,[最低價(成交)]
                        ,[收盤價(成交)]
                        ,[漲跌(成交)]
                        ,[漲幅(成交)(%)]

                    FROM [X01].[dbo].[可轉公司債收盤] where 日期 >= 20020101 order by 日期"""

        return df

In [2]:
sql_base = get_df()
sql_base.db_connect('X01')
sql_base.querys['CB'] = sql_base.make_CB()

Connecting to database using pyodbc...
Succesfully Connected to database using pyodbc!


In [3]:
CB行情表 = sql_base.load_data('CB')
CB行情表 = CB行情表.sort_values(by = ['日期','代號']).reset_index(drop = True)

In [4]:
#CB行情表

In [5]:
CB1 = pd.Series(CB行情表['代號'])

In [6]:
CBlist = CB1.sort_values().drop_duplicates()

In [7]:
CBlist = list(CBlist)

In [8]:
from tqdm import tqdm
data_dict = {str(CBlist[i]):CB行情表[CB行情表['代號'] == CBlist[i]] for i in tqdm(range(len(CBlist)))}


100%|██████████| 1829/1829 [01:55<00:00, 15.84it/s]


In [9]:
from tqdm import tqdm
data_pivots_ls = []
for feature in tqdm(CB行情表.columns[2:]):
    data_pivot = CB行情表.pivot('日期','代號',feature)
    data_pivots_ls.append(data_pivot)
# pivot_dict = {str(CBlist[i]):data_pivots_ls[i] for i in range(len(CBlist))}

# data_dict.update(pivot_dict)

100%|██████████| 31/31 [00:15<00:00,  2.00it/s]


In [10]:
pivot_dict = {str(CB行情表.columns[2:][i]):data_pivots_ls[i] for i in range(len(CB行情表.columns[2:]))}

data_dict.update(pivot_dict)

In [11]:
data_dict['99585'].tail(10)

,日期,代號,名稱,開盤價,最高價,最低價,收盤價,漲跌,漲幅(%),成交張數,...,成交張數(議價),成交筆數(議價),成交金額(議價)(千),RTIME,開盤價(成交),最高價(成交),最低價(成交),收盤價(成交),漲跌(成交),漲幅(成交)(%)
1144053,20231031,99585,世紀鋼五,117.000000,117.000000,108.699997,109.500000,-10.20,-8.52,101.0,...,NaN,NaN,NaN,468653409.0,117.000000,117.000000,108.699997,109.500000,-10.20,-8.52
1144348,20231101,99585,世紀鋼五,108.500000,109.800003,106.500000,109.800003,0.30,0.27,431.0,...,NaN,NaN,NaN,468753415.0,108.500000,109.800003,106.500000,109.800003,0.30,0.27
1144641,20231102,99585,世紀鋼五,110.199997,110.300003,109.000000,109.000000,-0.80,-0.73,49.0,...,NaN,NaN,NaN,468853406.0,110.199997,110.300003,109.000000,109.000000,-0.80,-0.73
1144934,20231103,99585,世紀鋼五,109.000000,109.150002,108.800003,109.150002,0.15,0.14,21.0,...,NaN,NaN,NaN,468953409.0,109.000000,109.150002,108.800003,109.150002,0.15,0.14
1145225,20231106,99585,世紀鋼五,109.500000,109.500000,109.199997,109.199997,0.05,0.05,3.0,...,NaN,NaN,NaN,469253408.0,109.500000,109.500000,109.199997,109.199997,0.05,0.05
1145516,20231107,99585,世紀鋼五,109.199997,109.199997,108.500000,109.199997,0.00,0.00,20.0,...,NaN,NaN,NaN,469353406.0,109.199997,109.199997,108.500000,109.199997,0.00,0.00
1145808,20231108,99585,世紀鋼五,109.500000,112.500000,109.500000,112.449997,3.25,2.98,127.0,...,12.0,1.0,1349.4,469455806.0,109.500000,112.500000,109.500000,112.449997,3.25,2.98
1146100,20231109,99585,世紀鋼五,112.500000,113.699997,112.500000,113.699997,1.25,1.11,85.0,...,NaN,NaN,NaN,469553409.0,112.500000,113.699997,112.500000,113.699997,1.25,1.11
1146394,20231110,99585,世紀鋼五,113.800003,117.400002,113.800003,117.099998,3.40,2.99,155.0,...,NaN,NaN,NaN,469653414.0,113.800003,117.400002,113.800003,117.099998,3.40,2.99
1146688,20231113,99585,世紀鋼五,117.500000,120.300003,117.000000,118.800003,1.70,1.45,395.0,...,NaN,NaN,NaN,469953406.0,117.500000,120.300003,117.000000,118.800003,1.70,1.45


In [12]:
data_pivots_ls[5]

代號,12101,12161,12171,12172,12173,12181,12251,12581,12582,13121,...,99461,99551,99552,99553,99581,99582,99583,99584,99585,99586
日期,,,,,,,,,,,,,,,,,,,,,
20060601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20060602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20060605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20060606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20060607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20231107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,-0.05
20231108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.25,3.35
20231109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.25,1.30
